# The Transaction-Level TinyALU

In [1]:
from pyuvm import *
from tb_pkg import *

class TinyAluTlm(uvm_component):
    @staticmethod
    def alu_op(A, B, op):
        assert isinstance(op, Ops), "The tinyalu op must be of type ops"
        if op == Ops.NOP:
            result =  A
        elif op == Ops.ADD:
            result = A + B
        elif op == Ops.AND:
            result = A & B
        elif op == Ops.XOR:
            result = A ^ B
        elif op == Ops.MUL:
            result = A * B
        time.sleep(0.1)
        return result
    
    def build_phase(self):
        self.stim_f = uvm_tlm_fifo("stim_f", self)
        self.cmd_f = uvm_tlm_analysis_fifo("cmd_f", self)
        self.result_f = uvm_tlm_analysis_fifo("result_f", self)

        self.stim_put = uvm_put_port("stim_put", self)
        self.stim_get = uvm_get_port("stim_get", self)
        
        self.cmd_put = uvm_put_port("cmd_put", self)
        self.cmd_get = uvm_get_port("cmd_get", self)
        
        self.result_put = uvm_put_port("result_put",self)
        self.result_get = uvm_get_port("result_get", self)

    def connect_phase(self):
        self.stim_put.connect(self.stim_f.put_export)
        self.stim_get.connect(self.stim_f.get_export)
        
        self.cmd_put.connect(self.cmd_f.put_export)
        self.cmd_get.connect(self.cmd_f.get_export)
        
        self.result_put.connect(self.result_f.put_export)
        self.result_get.connect(self.result_f.get_export)

    def send_op(self, op):
        self.stim_put.put(op)
    
    def get_cmd(self):
        cmd  = self.cmd_get.get()
        return cmd
    
    def get_result(self):
        result = self.result_get.get()
        return result        
        
    def run_phase(self):
        while True:
            cmd = self.stim_get.get()
            result = self.alu_op(cmd.A, cmd.B, cmd.op)
            result_txn = AluResult("result_txn", result)
            self.cmd_put.put(cmd)
            self.result_put.put(result_txn)

class TxnPrinter(uvm_subscriber):
    def write(self, txn):
        self.logger.info(txn)
    
class CmdMonitor(uvm_component):
    def __init__(self, name, parent, dut):
        super().__init__(name, parent)
        self.dut = dut
        
    def build_phase(self):
        self.analysis_port = uvm_analysis_port("analysis_port", self)
    
    def run_phase(self):
        cmd = self.dut.get_cmd()
        self.analysis_port.write(cmd)

class ResultMonitor(uvm_component):
    def __init__(self, name, parent, dut):
        super().__init__(name, parent)
        self.dut = dut
    
    def build_phase(self):
        self.analysis_port = uvm_analysis_port("analysis_port", self)

    def run_phase(self):
        result = self.dut.get_result()
        self.analysis_port.write(result)    
        
    
class AluEnv(uvm_env):
    def __init__(self, name, parent,dut):
        super().__init__(name, parent)
        self.dut = dut
        
    def build_phase(self):
        self.cmd_mon = CmdMonitor("cmd_mon", self, self.dut)
        self.rslt_mon = ResultMonitor("rslt_mon", self, self.dut)
        self.txn_printer = TxnPrinter("txn_printer", self)
        
    def connect_phase(self):
        self.cmd_mon.analysis_port.connect(self.txn_printer)
        self.rslt_mon.analysis_port.connect(self.txn_printer)

class AluTest(uvm_test):
    def build_phase(self):
        self.dut = TinyAluTlm("dut", self)
        self.env = AluEnv("env", self, self.dut)
        
    def run_phase(self):
        self.raise_objection()
        cmd = AluCommand("cmd", 0x55, 0x22, Ops.ADD)
        self.dut.send_op(cmd)
        time.sleep(0.5) # Wait for it to complete
        self.drop_objection()

uvm_root().run_test("AluTest")
        

INFO: <ipython-input-1-4ec1abee60cb>(77)[uvm_test_top.env.txn_printer]: cmd : A: 0x55 OP: ADD (1) B: 0x22
INFO: <ipython-input-1-4ec1abee60cb>(77)[uvm_test_top.env.txn_printer]: result_txn: 0x0077
